### MVP - Construção de um pipeline de dados

  A pipeline consistem em cinco etapas bem definidas, como buscar, coletar, modelar, carregar e análisar os dados.

  Na primeira etapa que é a busca dos dados, foi escolhido dados abertos na web no site Portal da Transparencia do governo federal [https://portaldatransparencia.gov.br/].

  Dentre as inumeras bases de dados no site, a escolhida foram os dados dos aposentados do Banco Central do Brasil (BACEN) que encontram-se ativos.  
  O período de análise será de Janeiro de 2025 até Setembro de 2025.

  A proposta é conseguir responder as seguintes perguntas com a massa de dados escolhida:
  - Qual a faixa etária dos servidores?
  - Qual é a média de idade desses servidores?   
  - Quantos desses servidores estão ocupando cargos de alta gestão ou função comissionada? 
  - Qual foi o fluxo de aposentados na ativa no período? 
  - Quantos aposentados foram reativados a cada mês? 
  - Em quais departamentos a concentração de aposentados na ativa é maior? 

  A segunda etapa consistem em coletar esses dados do site e importar ele dentro da ferramaneta databricks.


Paraparação para a camada 

In [0]:

%sql

-- Cria o catálogo principal para todos os seus dados do BACEN
CREATE CATALOG IF NOT EXISTS catalog_bacen
COMMENT 'Catálogo para dados públicos dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- Opcional: Crie os esquemas (databases) Bronze, Silver e Gold dentro do novo catálogo
CREATE SCHEMA IF NOT EXISTS catalog_bacen.bronze;
CREATE SCHEMA IF NOT EXISTS catalog_bacen.silver;
CREATE SCHEMA IF NOT EXISTS catalog_bacen.gold;

-- 1. Criação do Volume para armazenar os arquivos
-- Assumimos o nome 'dados_servidores' para o Volume que armazenará seus arquivos.
CREATE VOLUME IF NOT EXISTS catalog_bacen.bronze.dados_servidores
COMMENT 'Volume para arquivos CSV brutos dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- 2. Criação do Volume para armazenar as tabelas 
-- Assumimos o nome 'dados_servidores' para o Volume que armazenará suas tabelas.
CREATE VOLUME IF NOT EXISTS catalog_bacen.silver.dados_servidores
COMMENT 'Volume para armazenar as tabelas dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- 2. Criação do Volume para armazenar o modelo estrela
-- Assumimos o nome 'dados_servidores' para o Volume que armazenará o modelo estrela.
CREATE VOLUME IF NOT EXISTS catalog_bacen.gold.dados_servidores
COMMENT 'Volume para armazenar o modelo estrela dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- Define o catálogo recém-criado como o padrão para as próximas operações
USE CATALOG catalog_bacen;

## Camada Bronze

In [0]:
# Tente rodar esta linha em uma célula separada antes do código principal
%pip install unidecode

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests
import os
from zipfile import ZipFile
import shutil
import pandas as pd
import re
from unidecode import unidecode 
from pyspark.sql.functions import col

# --- CONFIGURAÇÃO DE CAMINHOS ---
UC_VOLUME_PATH = "/Volumes/catalog_bacen/bronze/dados_servidores/"

# --- FUNÇÃO DE LIMPEZA (MANTIDA) ---
def limpar_nome_coluna(nome_coluna):
    nome_limpo = unidecode(nome_coluna) 
    nome_limpo = re.sub(r' \(R\$\)(\(\*\))?$', '_REAIS', nome_limpo)
    nome_limpo = re.sub(r' \(U\$\)(\(\*\))?$', '_DOLAR', nome_limpo)
    nome_limpo = re.sub(r'[/,;{}\(\)\n\t\=–]', ' ', nome_limpo)
    nome_limpo = nome_limpo.strip().lower() 
    nome_limpo = re.sub(r'\s+', '_', nome_limpo)
    nome_limpo = nome_limpo.strip('_')
    nome_limpo = re.sub(r'__+', '_', nome_limpo)
    return nome_limpo

# --- LOOP PARA OS MESES (JANEIRO A SETEMBRO) ---
# Geramos uma lista de meses formatados: ['202501', '202502', ..., '202509']
meses_para_processar = [f"2025{str(mes).zfill(2)}" for mes in range(1, 10)]

print(f"Iniciando processamento para os meses: {meses_para_processar}")

for competencia in meses_para_processar:
    print(f"\n" + "="*50)
    print(f"PROCESSANDO COMPETÊNCIA: {competencia}")
    print("="*50)

    # Dinamiza as variáveis com base no mês
    ZIP_URL = f"https://portaldatransparencia.gov.br/download-de-dados/servidores/{competencia}_Aposentados_BACEN/"
    ZIP_FILENAME = f"{competencia}_Aposentados_BACEN.zip"
    TEMP_DOWNLOAD_PATH = f"/tmp/{ZIP_FILENAME}"
    TEMP_UNZIP_DIR = f"/tmp/unzip_{competencia}"

    try:
        # A. DOWNLOAD
        print(f" -> Baixando: {ZIP_URL}")
        response = requests.get(ZIP_URL, stream=True)
        
        if response.status_code == 404:
            print(f" [AVISO] Dados para {competencia} não encontrados (404). Pulando...")
            continue
            
        response.raise_for_status()
        
        with open(TEMP_DOWNLOAD_PATH, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024 * 1024):
                file.write(chunk)
                
        # B. DESCOMPACTAR
        os.makedirs(TEMP_UNZIP_DIR, exist_ok=True)
        with ZipFile(TEMP_DOWNLOAD_PATH, 'r') as zip_ref:
            zip_ref.extractall(TEMP_UNZIP_DIR)
        
        # C. PROCESSAR ARQUIVOS
        for filename in os.listdir(TEMP_UNZIP_DIR):
            if filename.lower().endswith('.csv'):
                local_csv_path = os.path.join(TEMP_UNZIP_DIR, filename)
                remote_delta_path = UC_VOLUME_PATH + filename.replace(".csv", "") + "_delta"
                
                print(f"    -> Lendo CSV: {filename}")
                
                # Lendo com tratamento de encoding
                with open(local_csv_path, 'r', encoding='ISO-8859-1') as f:
                    csv_content = f.read()

                temp_csv_file_on_volume = f"{UC_VOLUME_PATH}temp_{competencia}_{filename}"
                dbutils.fs.put(temp_csv_file_on_volume, csv_content, overwrite=True)
                
                # Spark Read
                df_temp = (spark.read 
                           .format("csv") 
                           .option("header", "true") 
                           .option("delimiter", ";") 
                           .option("encoding", "ISO-8859-1") 
                           .load(temp_csv_file_on_volume)
                          )
                
                # Renomear Colunas
                mapeamento = {c: limpar_nome_coluna(c) for c in df_temp.columns}
                df_limpo = df_temp
                for old_name, new_name in mapeamento.items():
                    df_limpo = df_limpo.withColumnRenamed(old_name, new_name)
                
                # Salvar Delta
                df_limpo.write.format("delta").mode("overwrite").save(remote_delta_path)
                dbutils.fs.rm(temp_csv_file_on_volume)
                print(f"    -> Sucesso: {remote_delta_path}")

        # D. LIMPEZA TEMPORÁRIA (por mês)
        os.remove(TEMP_DOWNLOAD_PATH) 
        shutil.rmtree(TEMP_UNZIP_DIR)
        print(f" -> Concluído com sucesso para {competencia}")

    except Exception as e:
        print(f" [ERRO] Falha ao processar {competencia}: {e}")
        # Continua para o próximo mês mesmo se um falhar
        continue

print("\nPipeline finalizado para todos os meses disponíveis.")

Iniciando processamento para os meses: ['202501', '202502', '202503', '202504', '202505', '202506', '202507', '202508', '202509']

PROCESSANDO COMPETÊNCIA: 202501
 -> Baixando: https://portaldatransparencia.gov.br/download-de-dados/servidores/202501_Aposentados_BACEN/
    -> Lendo CSV: 202501_Remuneracao.csv
Wrote 1722463 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Remuneracao_delta
    -> Lendo CSV: 202501_Observacoes.csv
Wrote 59 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Observacoes_delta
    -> Lendo CSV: 202501_Cadastro.csv
Wrote 1779125 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Cadastro_delta
 -> Concluído com sucesso para 202501

PROCESSANDO COMPETÊNCIA: 202502
 -> Baixando: https://portaldatransparencia.gov.br/download-de-dados/servidores/202502_Aposentados_BACEN/
    -> Lendo CSV: 202502_Cadastro.csv
Wrote 1777220 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_serv

## Camada Silver

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, current_timestamp, lit
from pyspark.sql import functions as F

# --- CONFIGURAÇÃO DE CAMINHOS ---
UC_VOLUME_PATH = "/Volumes/catalog_bacen/bronze/dados_servidores/"
PATH_SILVER = "/Volumes/catalog_bacen/silver/servidores_consolidado"

print(f"1. Localizando tabelas na Bronze...")

# Lista todos os arquivos/pastas no volume
files = dbutils.fs.ls(UC_VOLUME_PATH)

# Filtra apenas as pastas que terminam com '_delta'
paths_to_load = [f.path for f in files if f.path.endswith("_delta/") or "_delta" in f.path]

if not paths_to_load:
    raise Exception(f"Nenhuma tabela Delta encontrada em {UC_VOLUME_PATH}. Verifique se a camada Bronze rodou com sucesso.")

print(f"   -> Encontradas {len(paths_to_load)} pastas. Carregando e unificando...")

# Lê a primeira pasta para iniciar o DataFrame e faz Union com as demais
df_bronze = spark.read.format("delta").load(paths_to_load[0])

for path in paths_to_load[1:]:
    df_proximo = spark.read.format("delta").load(path)
    df_bronze = df_bronze.unionByName(df_proximo, allowMissingColumns=True)

# --- 2. TRANSFORMAÇÕES (LIMPEZA FINANCEIRA) ---
print("2. Iniciando limpeza financeira...")
cols_financeiras = [c for c in df_bronze.columns if '_reais' in c or '_dolar' in c]

df_silver = df_bronze

for c in cols_financeiras:
    # Tratamento para converter "1.234,56" em 1234.56 (Double)
    df_silver = df_silver.withColumn(
        c, 
        F.regexp_replace(F.regexp_replace(F.col(c), r'\.', ''), ',', '.').cast("double")
    )

# --- 3. DATA E AUDITORIA ---
df_silver = df_silver.withColumn(
    "data_referencia", 
    F.to_date(F.concat(F.lit("01/"), F.col("mes_referencia")), "dd/MM/yyyy")
).withColumn("etl_load_timestamp", F.current_timestamp())

# --- 4. GRAVAÇÃO ---
print(f"3. Gravando camada Silver consolidada em: {PATH_SILVER}")
# Criamos o diretório silver se não existir
dbutils.fs.mkdirs("/Volumes/catalog_bacen/silver/")
                   

df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(PATH_SILVER)

print("✅ Camada Silver pronta para análise!")

1. Localizando tabelas na Bronze...
   -> Encontradas 27 pastas. Carregando e unificando...
2. Iniciando limpeza financeira...
3. Gravando camada Silver consolidada em: /Volumes/catalog_bacen/silver/servidores_consolidado


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-8131620496492578>, line 50
     48 print(f"3. Gravando camada Silver consolidada em: {PATH_SILVER}")
     49 # Criamos o diretório silver se não existir
---> 50 dbutils.fs.mkdirs("/Volumes/catalog_bacen/silver/")
     52 df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(PATH_SILVER)
     54 print("✅ Camada Silver pronta para análise!")

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:54, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     51 class ExecutionError(Exception):
     52     pass
---> 54 raise ExecutionError(str(e)) from None

ExecutionError: Entry must be a full volume path. Path '/Volumes/catalog_bacen/silver' is incomplete. Expected format: /Volumes/<catalog>/<schema>/<volume>

JVM stacktrace:
com.databric